<a href="https://colab.research.google.com/github/iXiphos/DataMiningMath/blob/main/OneHot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

university_towns = []

with open('/content/university_towns.txt') as file:
  for line in file:
    if '[edit]' in line:
      state = line 
    else:
      university_towns.append((state, line))

university_towns[:10]

towns_df = pd.DataFrame(university_towns, columns=['State', 'RegionName'])  

FlickrData = pd.read_csv('https://github.com/realpython/python-data-cleaning/raw/master/Datasets/BL-Flickr-Images-Book.csv')

FlickrData['Edition Statement'].isna().value_counts(dropna=False)/len(FlickrData)

to_drop = ['Edition Statement','Corporate Author','Corporate Contributors','Former owner','Engraver']

FlickrData.drop(columns=to_drop, inplace=True)

FlickrData.set_index('Identifier', inplace=True)

regex = r'^(\d{4})'

FlickrDataExtr = FlickrData['Date of Publication'].str.extract(regex, expand=False)

FlickrData['Date of Publication'] = pd.to_numeric(FlickrDataExtr)

FlickrData['Date of Publication'].isnull().sum()/len(FlickrData)

FlickerDataPop = FlickrData['Place of Publication']

london = FlickerDataPop.str.contains('London')

oxford = FlickerDataPop.str.contains('Oxford')

FlickrData['Place of Publication'] = np.where(london, 'London',
         np.where(oxford, 'Oxford',
                  FlickerDataPop.str.replace('-', ' ')))

FlickrData.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Issuance type,Flickr URL,Shelfmarks
Identifier,,,,,,,,,
206,London,1879.0,S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
216,London,1868.0,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.
218,London,1869.0,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12625.dd.1.
472,London,1851.0,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 10369.bbb.15.
480,London,1857.0,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 9007.d.28.


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#Note on the bottom here, I misplaced the file and accidentally ran it so it has errors, with correct file it should run properly

university_towns = []

with open('/content/university_towns.txt') as file:
  for line in file:
    if '[edit]' in line:
      state = line 
    else:
      university_towns.append((state, line))

university_towns[:10]

towns_df = pd.DataFrame(university_towns, columns=['State', 'RegionName'])

#towns_df['State'] = towns_df['State'].str.replace('\[edit\]\\n', '')

def get_citystate(item):
  if ')' in item:
    return item[:item.find(')')+1]
  elif '[' in item:
    return item[:item.find('[')]
  else:
    return item

towns_df = towns_df.applymap(get_citystate)
new = towns_df['RegionName'].str.split('(',n = 1, expand = True)

towns_df['City'] = new[0]

towns_df['University'] = new[1].str.replace(')','')

towns_df.drop(columns = ['RegionName'], inplace = True)

towns_df['State'] = towns_df['State'].astype(str)
towns_df['City'] = towns_df['City'].astype(str)
towns_df['University'] = towns_df['University'].astype(str)

label_encoder = LabelEncoder()

towns_df['State'] = label_encoder.fit_transform(towns_df['State']) + 1

data_column_category = towns_df.select_dtypes(exclude=[np.number]).columns
#data_column_numeric = towns_df.select_dtypes(include=[np.number]).columns

scalecolumn = np.log10(towns_df['State'])

onehot = OneHotEncoder(sparse=False)

onehot = onehot.fit_transform(towns_df[data_column_category])

onehot_frame = pd.DataFrame(onehot, columns = onehot.get_feature_names(data_column_category))

final_df = pd.concat([scalecolumn ,onehot_encoded_frame, towns_df[data_column_category]], axis=1)

Features = onehot.get_feature_names(data_column_category)
TargetFeature = "State"
X_train, X_test, Y_train, Y_test = train_test_split(final_df[DATAFeatures], final_df['State'], test_size=.25)

#Best Model
xb = np.array(X_train)
#xb = xb.reshape(-1,1)
yb = np.array(Y_train)
yb = yb.reshape(-1,1)

xb_test = np.array(X_test)
#xb_test = xb.reshape(-1,1)
yb_test = np.array(Y_test)
yb_test = yb_test.reshape(-1,1)

bestModel = lm.LinearRegression().fit(xb,yb)
BMscore = bestModel.score(xb_test, yb_test)

print(final_df.info())
print('R^2:',BMscore)

#Questions
#Note on the bottom here, I misplaced the file and accidentally ran it so it has errors, with correct file it should run properly
#Feature = City & University
#Target = State
#Problem with One Hot is all data in the columns become 1 or 0 but also a good thing depending
#There is no different between scaled and non scaled
#You can not get accurate linear regression

AttributeError: ignored